# Bereinigung der Datensätze

## 1. Importieren der Pakete

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

## 2. Kundendaten

### 2.1 Kundendatei einlesen

In [ ]:
kunden = pd.read_csv("kunden.csv", decimal = '.', sep = ";")
kunden.head()

### 2.2 Überblick über Datentypen

In [ ]:
kunden.dtypes

### 2.3. Bereinigung

### 2.3.1 Fehlende Datenpunkte ermitteln und ersetzen

In [ ]:
kunden.isnull().sum()

In [ ]:
# Finden des Modalwertes:

In [ ]:
modalwert_kunden = kunden['Geschlecht'].mode()[0]

In [ ]:
modalwert_kunden

In [ ]:
# Ersetzen der 5 fehlenden Geschlechtseinträge mit dem Modalwert 1.0

In [ ]:
kunden.fillna(value = modalwert_kunden, axis = 0, inplace = True)

In [ ]:
# Überprüfung

In [ ]:
kunden.isnull().sum()

### 2.3.2 Ausreißer ermitteln und entfernen

In [ ]:
# Außreißer ermitteln
kunden.describe().T

### 2.3.2.1 Alter 

In [ ]:
boxplot_alter = kunden.boxplot(column = 'Alter')
plt.show()

In [ ]:
# zwei (?) Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR
Q1_Alter = kunden['Alter'].quantile(0.25)
Q3_Alter = kunden['Alter'].quantile(0.75)
IQR_Alter = Q3_Alter - Q1_Alter
# Berechnung der Ausreißer-Grenzen
untere_grenze_Alter = Q1_Alter - 1.5 * IQR_Alter
obere_grenze_Alter = Q3_Alter + 1.5 * IQR_Alter
kunden['Ausreißer'] = (kunden['Alter'] < untere_grenze_Alter) | (kunden['Alter'] > obere_grenze_Alter)
ausreißer_alter = kunden.Ausreißer.value_counts()
ausreißer_alter

In [ ]:
# Ermitteln der genauen Werte der Ausreißer

In [ ]:
kunden.loc[kunden['Ausreißer'] == True]

In [ ]:
# Personen im Alter von 100 und 120 Jahren werden kein Auto mehr kaufen

In [ ]:
# Ersetzen der Ausreißer in der Alters-Spalte durch Medianwert
alter_median = kunden['Alter'].median()
kunden.replace([120, 100], alter_median, inplace=True)

In [ ]:
# Überprüfen ob noch Ausreißer vorhanden sind
kunden['Ausreißer'] = (kunden['Alter'] < untere_grenze_Alter) | (kunden['Alter'] > obere_grenze_Alter)
ausreißer_alter = kunden.Ausreißer.value_counts()
ausreißer_alter

In [ ]:
# keine Ausreißer mehr vorhanden

## 2.3.2.2 Einkommen 

In [ ]:
boxplot_einkommen = kunden.boxplot(column = 'Einkommen')
plt.show()

In [ ]:
# ?? Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR der Einkommens-Spalte
Q1_einkommen = kunden['Einkommen'].quantile(0.25)
Q3_einkommen = kunden['Einkommen'].quantile(0.75)
IQR_einkommen = Q3_einkommen - Q1_einkommen
IQR_einkommen
untere_grenze_einkommen = Q1_einkommen - 1.5 * IQR_einkommen
obere_grenze_einkommen = Q3_einkommen + 1.5 * IQR_einkommen
kunden['Ausreißer'] = (kunden['Einkommen'] < untere_grenze_einkommen) | (kunden['Einkommen'] > obere_grenze_einkommen)
ausreißer_einkommen = kunden.Ausreißer.value_counts()
ausreißer_einkommen

In [ ]:
# Ermitteln der genauen Werte der Ausreißer
kunden.loc[kunden['Ausreißer'] == True]

In [ ]:
# Ersetzen der Ausreißer in der Einkommens-Spalte durch Medianwert
einkommen_median = kunden['Einkommen'].median()
kunden.replace({'Einkommen': [1.000000e+09, 1.076890e+05, 0.000000e+00, -62840.374523]}, einkommen_median, inplace=True)

In [ ]:
# Überprüfen ob noch Ausreißer vorhanden sind
kunden['Ausreißer'] = (kunden['Einkommen'] < untere_grenze_einkommen) | (kunden['Einkommen'] > obere_grenze_einkommen)
ausreißer_einkommen = kunden.Ausreißer.value_counts()
ausreißer_einkommen

In [ ]:
# keine Ausreißer mehr vorhanden

### 2.3.2.3 Preis

In [ ]:
boxplot_preis = kunden.boxplot(column = 'Preis')
plt.show()

In [ ]:
# ein (?) Ausreißer zu erkennen

In [ ]:
Q1_preis = kunden['Preis'].quantile(0.25)
Q3_preis = kunden['Preis'].quantile(0.75)
IQR_preis = Q3_preis - Q1_preis
IQR_preis
untere_grenze_preis = Q1_preis - 1.5 * IQR_preis
obere_grenze_preis = Q3_preis + 1.5 * IQR_preis
kunden['Ausreißer'] = (kunden['Preis'] < untere_grenze_preis) | (kunden['Preis'] > obere_grenze_preis)
ausreißer_preis = kunden.Ausreißer.value_counts()
ausreißer_preis

In [ ]:
# Ermitteln der genauen Werte der Ausreißer
kunden.loc[kunden['Ausreißer'] == True]

In [ ]:
# bei einem Einkommen von 59.013 ist es überraschend sich ein Auto im Preis von 165.482,31 zu kaufen, aber möglich, daher wird der Wert behalten


### 2.3.2.4 Zeit

In [ ]:
boxplot_preis = kunden.boxplot(column = 'Zeit')
plt.show()

In [ ]:
# mehrere Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR der Zeit-Spalte
Q1_zeit = kunden['Zeit'].quantile(0.25)
Q3_zeit = kunden['Zeit'].quantile(0.75)
IQR_zeit = Q3_zeit - Q1_zeit
IQR_zeit
untere_grenze_zeit = Q1_zeit - 1.5 * IQR_zeit
obere_grenze_zeit = Q3_zeit + 1.5 * IQR_zeit
kunden['Ausreißer'] = (kunden['Zeit'] < untere_grenze_zeit) | (kunden['Zeit'] > obere_grenze_zeit)
ausreißer_zeit = kunden.Ausreißer.value_counts()
ausreißer_zeit

In [ ]:
# Ermittlung der genauen Werte der Ausreißer
kunden.loc[kunden['Ausreißer'] == True]

In [ ]:
# Die Zeiten sind möglich, z.B. 12 Minuten weil keine Probefahrt gemacht wurde oder 80 Minuten weil eine lange gemacht wurde, Werte bleiben erhalten

In [ ]:
kunden

In [ ]:
# Boolean-Spalte wieder entfernen
kunden = kunden.drop(kunden.columns[-1], axis=1)
kunden

## 3. Besucherdaten

### 3.1 Besucherdatei einlesen

In [ ]:
besucher = pd.read_csv("besucher.csv", decimal = '.', sep = ";")
besucher.head()

### 3.2 Überblick über Datentypen

In [ ]:
besucher.dtypes

In [ ]:
# zum Vergleich: Kunden
kunden.dtypes

In [ ]:
# Alter = int, Einkommen = float (ändern bei besucher), Preis = float, Geschlecht = int (ändern bei kunden), Zeit = float (ändern bei besucher), KundeNr = object

### 3.3. Bereinigung

In [ ]:
# vermutlich ist die Kommaschreibweise aus dem deutschen, während wir die Werte in der englischen Schreibweise mit einem Punkt, statt Komma benötigen

### 3.3.1 Bereinigung der Nachkommastellen und Umwandlung der dtypes

### 3.3.1.1 Einkommen

In [ ]:
besucher["Einkommen"] = besucher["Einkommen"].str.replace(",",".")
besucher["Einkommen"] = pd.to_numeric(besucher["Einkommen"]) 
besucher["Einkommen"] = besucher["Einkommen"].round(0).astype("float64")
besucher

### 3.3.1.2 Zeit

In [ ]:
besucher["Zeit"] = besucher["Zeit"].str.replace(",",".")
besucher["Zeit"] = pd.to_numeric(besucher["Zeit"])
besucher["Zeit"] = besucher["Zeit"].round(0).astype("float64")
besucher

### 3.3.1.3 Geschlecht

In [ ]:
kunden["Geschlecht"] = kunden["Geschlecht"].astype("int64")
kunden

In [ ]:
# Überprüfen der dtypes
besucher.dtypes, kunden.dtypes

### 3.3.2 Fehlende Datenpunkte ermitteln und ersetzen

In [ ]:
besucher.isnull().sum()

In [ ]:
# keine fehlenden Datenpunkte im Datensatz

### 3.3.3 Ausreißer ermitteln und entfernen

In [ ]:
# Außreißer ermitteln
kunden.describe().T

### 3.3.3.1 Alter

In [ ]:
boxplot_besucher_alter = besucher.boxplot(column = 'Alter')
plt.show()

In [ ]:
# sowohl bei describe-Tabelle als auch beim boxplot keine Ausreißer zu erkennen

### 3.3.3.2 Einkommen

In [ ]:
boxplot_besucher_einkommen = besucher.boxplot(column = 'Einkommen')
plt.show()

In [ ]:
# mehrere Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR der Zeit-Spalte
Q1_einkommen_besucher = besucher['Einkommen'].quantile(0.25)
Q3_einkommen_besucher = besucher['Einkommen'].quantile(0.75)
IQR_einkommen_besucher = Q3_einkommen_besucher - Q1_einkommen_besucher
untere_grenze_einkommen_besucher = Q1_einkommen_besucher - 1.5 * IQR_einkommen_besucher
obere_grenze_einkommen_besucher = Q3_einkommen_besucher + 1.5 * IQR_einkommen_besucher
besucher['Ausreißer'] = (besucher['Einkommen'] < untere_grenze_einkommen_besucher) | (besucher['Einkommen'] > obere_grenze_einkommen_besucher)
ausreißer_einkommen_besucher = besucher.Ausreißer.value_counts()
ausreißer_einkommen_besucher

In [ ]:
# Ermitteln der genauen Werte der Ausreißer
besucher.loc[besucher['Ausreißer'] == True]

In [ ]:
# so niedriges und so hohes Einkommen ist möglich, daher bleiben Werte erhalten

### 3.3.3.3 Zeit

In [ ]:
boxplot_besucher_zeit = besucher.boxplot(column = 'Zeit')
plt.show()

In [ ]:
# mehrere Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR der Zeit-Spalte
Q1_zeit_besucher = besucher['Zeit'].quantile(0.25)
Q3_zeit_besucher = besucher['Zeit'].quantile(0.75)
IQR_zeit_besucher = Q3_zeit_besucher - Q1_zeit_besucher
untere_grenze_zeit_besucher = Q1_zeit_besucher - 1.5 * IQR_zeit_besucher
obere_grenze_zeit_besucher = Q3_zeit_besucher + 1.5 * IQR_zeit_besucher
besucher['Ausreißer'] = (besucher['Zeit'] < untere_grenze_zeit_besucher) | (besucher['Zeit'] > obere_grenze_zeit_besucher)
ausreißer_zeit_besucher = besucher.Ausreißer.value_counts()
ausreißer_zeit_besucher

In [ ]:
# Ermittlung der genauen Werte der Ausreißer
besucher.loc[besucher['Ausreißer'] == True]

In [ ]:
# so kurze und lange Zeiten sind möglich, daher bleiben Werte erhalten

In [ ]:
besucher

In [ ]:
# Boolean-Spalte wieder entfernen

In [ ]:
besucher = besucher.drop(besucher.columns[-1], axis=1)
besucher

## 4. Geodaten

### 4.1. Geodatei einlesen

In [ ]:
geo = pd.read_csv("geo.txt", sep = "\t")
geo.head()

## 4.2 Überblick über Datentypen

In [ ]:
geo.dtypes

In [ ]:
# bei kunden und besucher ist der dypte der KundeNr auch object, daher keine Änderung benötigt

### 4.3 Bereinigung

### 4.3.1 Fehlende Datenpunkte ermitteln und ersetzen

In [ ]:
geo.isnull().sum()

### 4.3.2 Ausreißer ermitteln und entfernen/ersetzen

In [ ]:
# Übersicht aller Niederlassungen
geo_niederlassung_uniques = geo["Niederlassung"].unique()
print(geo_niederlassung_uniques)

In [ ]:
# Bereinigung der Datenpunkte (NRW, Düsseldorf zu Nordrhein-Westfalen), (Berlin-Charlottenburg, Berlin-Mitte, BERLIN zu Berlin)
geo_cleaning = {
    "NRW" : "Nordrhein-Westfalen" , 
    "Berlin-Charlottenburg" : "Berlin" ,
    "Berlin-Mitte" : "Berlin" ,
    "BERLIN" : "Berlin",
    "Düsseldorf" : "Nordrhein-Westfalen"}

geo["Niederlassung"] = geo["Niederlassung"].replace(geo_cleaning, regex = True)

In [ ]:
# Überprüfung
geo_niederlassung_uniques2 = geo["Niederlassung"].unique()
print(geo_niederlassung_uniques2)

## 5. Zusammenführung der Listen

### 5.1 Kunden und Besucher mit Geo

### 5.1.1 Kunden mit Geo 

In [ ]:
kunden_geo = pd.merge(kunden,geo, on = "KundeNr")
kunden_geo

### 5.1.2 Besucher mit Geo

In [ ]:
besucher_geo = pd.merge(besucher, geo, on = "KundeNr")
besucher_geo

### 5.2 kunden_geo und besucher_geo

In [ ]:
kunden_besucher_geo = pd.concat([kunden_geo, besucher_geo], axis = 0, ignore_index = True)
kunden_besucher_geo

## 6. Fragen

### Frage 1: Wie viele Autos wurden verkauft?

In [ ]:
anzahl_verkaufter_autos = len(kunden_geo)
anzahl_verkaufter_autos

In [ ]:
autos_verkauft = kunden_geo.shape[0]
autos_verkauft

In [ ]:
# Antwort zu Frage 1: 1104

### Frage 2: Was ist der Höchst-, Mindest- und Durchschnittspreis?

In [ ]:
max_preis = kunden_geo["Preis"].max()
min_preis = kunden_geo["Preis"].min()
durchschnitt_preis = kunden_geo["Preis"].mean()

max_preis, min_preis, durchschnitt_preis

In [ ]:
# Antwort zu Frage 2: Höchstpreis = 165.482,31, Mindestpreis = 21.471,65 und Durchschnittspreis = 88.975,20

### Frage 3: Wie hoch war der Gesamtumsatz?

In [ ]:
gesamtumsatz = kunden_geo["Preis"].sum()
gesamtumsatz


In [ ]:
# Antwort zu Frage 3: Gesamtumsatz = 98.228.619,38 

In [ ]:
# Formattierung der Werte in EUR 
def form_eur(wert):
    return "{:,.2f}€".format(wert)

max_preis_eur = form_eur(max_preis)
min_preis_eur = form_eur(min_preis)
durchschnitt_preis_eur = form_eur(durchschnitt_preis)
gesamtumsatz_eur = form_eur(gesamtumsatz)
max_preis_eur

In [ ]:
# Erstellung einer Preisübersicht
Preisübersicht = pd.DataFrame({
    "Maximaler Verkaufspreis" : [max_preis_eur],
    "Minimaler Verkaufspreis" : [min_preis_eur],
    "Durchschnittspreis" : [durchschnitt_preis_eur],
    "Gesamtumsatz" : [gesamtumsatz_eur]
})
display(Preisübersicht)

### Frage 4: Wie viele AUtos wurden pro Bundesland verkauft?

In [ ]:
autos_pro_bundesland = kunden_geo["Niederlassung"].value_counts()
autos_pro_bundesland


In [ ]:
# Erstellen eines Diagramms zur Darstellung
# Dataframe
daten = {"Bundesland" : ["Baden-Württemberg" , "Bayern" , "Nordrhein-Westfalen" , "Berlin" , "Hessen" , "Niedersachsen",
                        "Sachsen" , "Thüringen" , "Hamburg" , "Brandenburg"] , 
        "Verkäufe" : [205 , 188 , 175 , 126 , 96 , 96 , 89 , 49 , 43 , 37]}

df_autos_pro_bundesland = pd.DataFrame(daten)

verkäufe_durchschnitt = df_autos_pro_bundesland["Verkäufe"].mean()

# Diagramm
plt.figure(figsize=(10, 6))
diagramm = plt.bar(df_autos_pro_bundesland["Bundesland"],df_autos_pro_bundesland["Verkäufe"], color="lightsteelblue")
plt.axhline(y=verkäufe_durchschnitt, color='red', linestyle='--', linewidth=2)

for bar in diagramm:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{int(height)}", ha = "center", va = "center_baseline", fontsize=10)
    
plt.title("Anzahl der verkauften Autos pro Bundesland")
plt.xlabel("Bundesland")
plt.ylabel("Anzahl der verkauften Autos")
plt.xticks(rotation=45, ha='right')  
plt.tight_layout()
plt.show()

In [ ]:
# Antwort zu Frage 4: siehe Diagramm

### Frage 5: Wie hoch war der durchschnittliche Umsatz (vermutlich ist Preis gemeint) pro Bundesland?

In [ ]:
umsatz_pro_bundesland = kunden_geo.groupby("Niederlassung")["Preis"].agg("mean")
umsatz_pro_bundesland = umsatz_pro_bundesland.round(2)
umsatz_pro_bundesland

In [ ]:
# Erstellen eines Diagramms zur Darstellung
# Dataframe
daten2 = {"Bundesland" : ["Baden-Württemberg" , "Bayern" , "Nordrhein-Westfalen" , "Berlin" , "Hessen" , "Niedersachsen",
                        "Sachsen" , "Thüringen" , "Hamburg" , "Brandenburg"] , 
        "Umsatz" : [90969.56 , 89344.31 , 89584.98 , 86673.64 , 85549.50 , 85864.93 , 87963.31 , 88633.27 , 89455.08 , 90816.99]}

df_umsatz_pro_bundesland = pd.DataFrame(daten2)

# Diagramm
plt.figure(figsize=(10, 6))
diagramm2 = plt.bar(df_umsatz_pro_bundesland["Bundesland"],df_umsatz_pro_bundesland["Umsatz"], color="lightsteelblue")

for bar in diagramm2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{int(height)}", ha = "center", va = "center_baseline", fontsize=10)

plt.title("Umsatz pro Bundesland")
plt.xlabel("Bundesland")
plt.ylabel("Umsatz (in €)")
plt.xticks(rotation=45, ha='right')  
plt.tight_layout()
plt.show()


In [ ]:
# Antwort zu Frage 5: siehe Diagramm

### Frage 6: In welchem Bundesland wurde das teuerste Auto verkauft?

In [ ]:
teuerstes_auto_bundesland = kunden_geo.loc[kunden_geo["Preis"].idxmax()]["Niederlassung"]
teuerstes_auto_bundesland

In [ ]:
# Antwort zu Frage 6: Das teuerste Auto wurde in Niedersachsen verkauft

### Frage 7: Haben mehr Frauen oder Männer unsere Autos gekauft?

In [ ]:
geschlecht_vergleich = kunden_geo["Geschlecht"].value_counts()

labels = ["männlich", "weiblich"]
sizes = [700,400]
colors = ["lightblue","lightpink"]
plt.pie(sizes, labels = labels, colors = colors, autopct = "%1.1f%%", startangle = 90)
plt.axis("equal")
plt.title("Verteilung der verkauften Autos nach Geschlecht")
plt.show()


In [ ]:
geschlecht_vergleich = kunden_geo["Geschlecht"].value_counts()
geschlecht_vergleich

In [ ]:
# Antwort zu Frage 7: Es haben mehr Männer unsere Autos gekauft

### Frage 8: Wie hoch ist das Durchschnittsalter unserer Kunden? Wie hoch ist das Durchschnittsalter unserer Besucher?

In [ ]:
durchschnittsalter_kunden = kunden_geo["Alter"].mean()
durchschnittsalter_besucher = besucher_geo["Alter"].mean()

durchschnittsalter_kunden, durchschnittsalter_besucher

In [ ]:
# Antwort zu Frage 8: Das Durchschnittsalter unserer Kunden ist 50,38 Jahre und das Durchschnittsalter unserer Besucher ist 49,88 Jahre

### Frage 9: Wie hoch ist das Durchschnittseinkommen unserer Kunden? Wie hoch ist das Durchschnittseinkommen unserer Besucher?

In [ ]:
durchschnittseinkommen_kunden = kunden_geo["Einkommen"].mean()
durchschnittseinkommen_besucher = besucher_geo["Einkommen"].mean()

durchschnittseinkommen_kunden, durchschnittseinkommen_besucher


In [ ]:
# Antwort zu Frage 9: Das Durchschnittseinkommen unserer Kunden ist 58.850,91 und das Durchschnittseinkommen unserer Besucher ist 59.367,93

In [ ]:
t_stat, p_val = stats.ttest_ind(kunden_geo["Einkommen"].dropna(), besucher_geo["Einkommen"].dropna(), equal_var=False)
t_stat, p_val
